In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
from statistics import mean
dataset = pd.read_csv('../input/train.csv')
test_dataset = pd.read_csv("../input/test.csv")
#check for nan values in dataset
print('Checking for NULLs in train dataset')
for item in dataset.keys():
    print("Nan in ",item," is ", dataset[item].isna().sum())
    
print('\nChecking for NULLs in test dataset')
for item in test_dataset.keys():
    print("Nan in ",item," is ", test_dataset[item].isna().sum())

In [ ]:
#assigning mean values of age for both male and female
age_gender = dataset[['Age', 'Sex']].dropna()
male_age_mean = mean(age_gender[age_gender['Sex'] == 'male']['Age'])
female_age_mean = mean(age_gender[age_gender['Sex'] == 'female']['Age'])
dataset.loc[(dataset['Sex']=='female') & (dataset['Age'].isna()), 'Age'] = female_age_mean
dataset.loc[(dataset['Sex']=='male') & (dataset['Age'].isna()), 'Age'] = male_age_mean
test_dataset.loc[(dataset['Sex']=='female') & (test_dataset['Age'].isna()), 'Age'] = female_age_mean
test_dataset.loc[(dataset['Sex']=='male') & (test_dataset['Age'].isna()), 'Age'] = male_age_mean


In [ ]:
#filling missing values of fare
test_dataset['Fare'].fillna(0, inplace = True)

In [ ]:
#filling missing values of embarked
for item in dataset.loc[(dataset['Embarked'].isna()), 'Fare']:
    if item > 50:
        dataset.loc[(dataset['Embarked'].isna()), 'Embarked'] = 'S'
    elif item > 25:
        dataset.loc[(dataset['Embarked'].isna()), 'Embarked'] = 'C'
    else:
        dataset.loc[(dataset['Embarked'].isna()), 'Embarked'] = 'Q'

In [ ]:
#checking for null values again ignoring cabin
print('Checking for NULLs in train dataset')
for item in dataset.keys():
    print("Nan in ",item," is ", dataset[item].isna().sum())
    
print('\nChecking for NULLs in test dataset')
for item in test_dataset.keys():
    print("Nan in ",item," is ", test_dataset[item].isna().sum())

In [ ]:
#dropping columns that are not so significant
dataset.drop(columns = ['Cabin','PassengerId'], inplace = True)
test_dataset.drop(columns = ['Cabin', 'PassengerId'], inplace = True)

In [ ]:
#Label Encoder on Sex
from sklearn.preprocessing import LabelEncoder
sex_encoder = LabelEncoder().fit(dataset['Sex'])
dataset['Sex'] = sex_encoder.transform(dataset['Sex'])
test_dataset['Sex'] = sex_encoder.transform(test_dataset['Sex'])

In [ ]:
#Label Encoder on Embarked
embarked_encoder = LabelEncoder().fit(dataset['Embarked'])
dataset['Embarked'] = embarked_encoder.transform(dataset['Embarked'])
test_dataset['Embarked'] = embarked_encoder.transform(test_dataset['Embarked'])

In [ ]:
dataset

In [ ]:
#collecting name titles

import numpy as np
def get_titles(n):
    names = np.array(n.apply(lambda x : x.split(',')).tolist())[:,1]
    names = [item.split(' ')[1] for item in names]
    return pd.Series(names)
dataset['Name_title'] = get_titles(dataset['Name'])
dataset.drop(columns=['Name'], inplace = True)
test_dataset['Name_title'] = get_titles(test_dataset['Name'])
test_dataset.drop(columns=['Name'], inplace=True)

In [ ]:
#encoding the name titles

print(dataset['Name_title'].value_counts())
print(test_dataset['Name_title'].value_counts())

name_survived = dataset.groupby(['Name_title'])['Survived'].mean()
name_survived_encoding = {}
for item in name_survived.index:
    if name_survived[item] == 0:
        name_survived_encoding[item] = 0
    elif name_survived[item] < 0.5:
        name_survived_encoding[item] = 1
    elif name_survived[item] == 1:
        name_survived_encoding[item] = 4
    elif name_survived[item] > 0.5:
        name_survived_encoding[item] = 3
    elif name_survived[item] == 0.5:
        name_survived_encoding[item] = 2
        
#Dona. not present in train dataset lets assume that its 0
for item in test_dataset['Name_title'].value_counts().index:
    if(item[0] == 'D' and item[1] == 'o'):
        name_survived_encoding[item] = 0

name_encoded_train = [name_survived_encoding[item] for item in dataset['Name_title']]
name_encoded_test = [name_survived_encoding[item] for item in test_dataset['Name_title']]


In [ ]:
#keeping the name_titles saved
Name_titles_train = dataset['Name_title']
Name_titles_test = test_dataset['Name_title']

#replacing name_title in dataset with encoding
dataset['Name_title'] = name_encoded_train
test_dataset['Name_title'] = name_encoded_test

In [ ]:
#extracting features from tickets
import numpy as np
ticket_list_train = np.array(dataset['Ticket'].tolist())
ticket_list_test = np.array(test_dataset['Ticket'].tolist())

#generating first letter of the ticket
ticket_list_letter_train = [item[0] for item in ticket_list_train]
ticket_list_letter_test = [item[0] for item in ticket_list_test]

#if the first character is changing to Na
for a,item in enumerate(ticket_list_letter_train):
    if(item.isdigit()):
        ticket_list_letter_train[a] = 'Na'
for a,item in enumerate(ticket_list_letter_test):
    if(item.isdigit()):
        ticket_list_letter_test[a] = 'Na'

#hot encoding the ticket
ticket_encoder = LabelEncoder().fit(ticket_list_letter_train)
ticket_encoded_train = ticket_encoder.transform(ticket_list_letter_train)
ticket_encoded_test = ticket_encoder.transform(ticket_list_letter_test)

#keeping the original tickets
Tickets_train = dataset['Ticket']
Tickets_test = test_dataset['Ticket']

dataset['Ticket'] = ticket_encoded_train
test_dataset['Ticket'] = ticket_encoded_test

In [ ]:
#family count 

dataset['Family_count'] = dataset['SibSp'] + dataset['Parch']
test_dataset['Family_count'] = test_dataset['SibSp'] + test_dataset['Parch']

In [ ]:
#Scaling the data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
drop_tables = ['Survived', 'SibSp']
train_data = dataset.drop(drop_tables, axis = 1)
target = dataset['Survived']
scaler.fit(dataset.drop(drop_tables, axis = 1))
train_transformed = scaler.transform(dataset.drop(drop_tables, axis = 1))
test_transformed = scaler.transform(test_dataset.drop(['SibSp'], axis=1))

In [ ]:
#sclaing test_train_data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train_data, target)
scaler = StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
kfolds = 5
best_score = 0
best_gamma = 0
best_c = 0
for c in [1, 10, 100, 200, 300]:
    for g in [0.01, 0.1, 1, 10]:
        model = SVC(kernel = 'rbf', gamma = g, C = c)
        scores = cross_val_score(model, X_train_transformed, Y_train, cv = kfolds)
        scores = np.mean(scores)
        if scores > best_score:
            best_score = scores
            best_gamma = g
            best_c = c
print("SVC gamma = ", best_gamma, " c = ", best_c)
best_model = SVC(kernel = 'rbf', gamma = best_gamma, C = best_c).fit(X_train_transformed, Y_train)
print("Accuracy by SVC is ", best_model.score(X_test_transformed, Y_test))

In [ ]:
test_original = pd.read_csv("../input/test.csv")

In [ ]:
svc_model = SVC(kernel = 'rbf', gamma=best_gamma, C = best_c).fit(train_transformed, target)
y_predict = svc_model.predict(test_transformed)
final_df = pd.DataFrame(test_original['PassengerId'], columns=['PassengerId'])
final_df['Survived'] = y_predict
final_df.to_csv('final_output_svm_dec4.csv', index = False)